In [2]:
import cv2
import math
import numpy as np

In [37]:
def get_frames_with_labels(video_filepath, event_timestamp_millis, sequence_length, frame_step):
    vid_cap = cv2.VideoCapture(video_filepath)

    if not vid_cap.isOpened():
        print("Error opening video")
        return None

    vid_frame_count = vid_cap.get(cv2.CAP_PROP_FRAME_COUNT)
    
    vid_fps = vid_cap.get(cv2.CAP_PROP_FPS)
    event_frame_index = math.floor(event_timestamp_millis / 1000 * vid_fps) # todo: revisit this if there is issue with indexing too early

    max_frame_steps_event_to_beginning = math.floor(event_frame_index / frame_step)
    max_frame_steps_event_to_end = math.floor((vid_frame_count - 1 - event_frame_index)/ frame_step) # reason for minus 1 is this is about index and not about frame count
    
    if max_frame_steps_event_to_end - max_frame_steps_event_to_beginning < sequence_length:
        print(f"Not possible for frame step {frame_step} and sequence length {sequence_length}")
        print(max_frame_steps_event_to_end - max_frame_steps_event_to_beginning)
        # return None

    min_start_frame_idx = max(event_frame_index - (frame_step * (sequence_length - 1)), 0)
    max_start_frame_idx = min(event_frame_index, (vid_frame_count - 1) - (sequence_length - 1) * frame_step)

    frame_indexes = []
    
    for start_frame_idx in range(min_start_frame_idx, (max_start_frame_idx + frame_step), frame_step):
        # generate the index
        frame_indexes.append([i for i in range(start_frame_idx, start_frame_idx + sequence_length * frame_step, frame_step)])

    frame_indexes = np.array(frame_indexes)
    labels = np.array(frame_indexes >= event_frame_index, dtype=np.int8)
    
    return frame_indexes, labels

# test
video_filepath = 'data/running_1.mp4'
# assert get_frames_with_labels(video_filepath, event_timestamp_millis=1000, sequence_length=40, frame_step=13) is not None
# assert get_frames_with_labels(video_filepath, event_timestamp_millis=1000, sequence_length=774, frame_step=13) is None
result = get_frames_with_labels(video_filepath, event_timestamp_millis=1000, sequence_length=58, frame_step=13)
print(result[0].shape)
print(result)

Not possible for frame step 13 and sequence length 58
55
(4, 58)
(array([[  0,  13,  26,  39,  52,  65,  78,  91, 104, 117, 130, 143, 156,
        169, 182, 195, 208, 221, 234, 247, 260, 273, 286, 299, 312, 325,
        338, 351, 364, 377, 390, 403, 416, 429, 442, 455, 468, 481, 494,
        507, 520, 533, 546, 559, 572, 585, 598, 611, 624, 637, 650, 663,
        676, 689, 702, 715, 728, 741],
       [ 13,  26,  39,  52,  65,  78,  91, 104, 117, 130, 143, 156, 169,
        182, 195, 208, 221, 234, 247, 260, 273, 286, 299, 312, 325, 338,
        351, 364, 377, 390, 403, 416, 429, 442, 455, 468, 481, 494, 507,
        520, 533, 546, 559, 572, 585, 598, 611, 624, 637, 650, 663, 676,
        689, 702, 715, 728, 741, 754],
       [ 26,  39,  52,  65,  78,  91, 104, 117, 130, 143, 156, 169, 182,
        195, 208, 221, 234, 247, 260, 273, 286, 299, 312, 325, 338, 351,
        364, 377, 390, 403, 416, 429, 442, 455, 468, 481, 494, 507, 520,
        533, 546, 559, 572, 585, 598, 611, 624, 637, 